### Import libraries

In [17]:
import numpy as np
import pandas as pd
from absl import flags
import tensorflow as tf
import tensorflow_privacy
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
%matplotlib inline
tf.get_logger().setLevel('ERROR')
tf.compat.v1.disable_v2_behavior()




### Load Dataset

In [8]:
# Loading cifar10 image dataset
data_train, data_test = cifar10.load_data()
x_train, y_train = data_train
x_test, y_test = data_test

# Normalizing pixel values of images
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flattening images in dataset
y_train = y_train.flatten().reshape(y_train.shape[0], 1)
y_test = y_test.flatten().reshape(y_test.shape[0], 1)

# One hot encoding of labels/target column
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

print("training:", x_train.shape, y_train.shape)
print("testing:", x_test.shape, y_test.shape)

training: (50000, 32, 32, 3) (50000, 10)
testing: (10000, 32, 32, 3) (10000, 10)


### Model1: Simple CNN

In [11]:
# Hyper-parameters
epochs = 20
batch_size = 500

l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 250
learning_rate = 0.25

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [12]:
# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 8,
                           strides=2,
                           padding='same',
                           activation='relu',
                           input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Conv2D(32, 4,
                           strides=2,
                           padding='valid',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [13]:
# define optimizer (dp-sgd) 
optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)
# define loss function
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [14]:
# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# Train model
model.fit(x_train, y_train,
          epochs=epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

Train on 50000 samples, validate on 10000 samples


2022-03-08 16:42:30.484258: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-08 16:42:31.402756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/20


2022-03-08 16:42:31.686466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-08 16:42:31.846137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - ETA: 0s - loss: 2.2188 - acc: 0.1664 

2022-03-08 17:04:09.816648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - 1299s 26ms/sample - loss: 2.2188 - acc: 0.1664 - val_loss: 2.1000 - val_acc: 0.2320
Epoch 2/20
50000/50000 [==============================] - 1223s 24ms/sample - loss: 2.0589 - acc: 0.2534 - val_loss: 1.9255 - val_acc: 0.3094
Epoch 3/20
50000/50000 [==============================] - 1212s 24ms/sample - loss: 1.9223 - acc: 0.3040 - val_loss: 1.8572 - val_acc: 0.3305
Epoch 4/20
50000/50000 [==============================] - 1217s 24ms/sample - loss: 1.8465 - acc: 0.3341 - val_loss: 1.8674 - val_acc: 0.3170
Epoch 5/20
50000/50000 [==============================] - 1208s 24ms/sample - loss: 1.7838 - acc: 0.3603 - val_loss: 1.7684 - val_acc: 0.3776
Epoch 6/20
50000/50000 [==============================] - 1221s 24ms/sample - loss: 1.7363 - acc: 0.3805 - val_loss: 1.7594 - val_acc: 0.3869
Epoch 7/20
50000/50000 [==============================] - 1227s 25ms/sample - loss: 1.6895 - acc: 0.4000 - val_loss: 1.7437 - val_acc: 0.3961
Epoch 8/20
50000/

In [15]:
# Calculate privacy performance
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=x_train.shape[0],
                                              batch_size=batch_size,
                                              noise_multiplier=noise_multiplier,
                                              epochs=epochs,
                                              delta=1e-5)

DP-SGD with sampling rate = 1% and noise_multiplier = 1.3 iterated over 2000 steps satisfies differential privacy with eps = 1.8 and delta = 1e-05.
The optimal RDP order is 11.0.


(1.798929515280597, 11.0)

### Model2: ResNet with transfer learning

In [26]:
# Hyper-parameters
epochs = 50
batch_size = 500

l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 250
learning_rate = 0.25

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [27]:
#Building a ResNet model with pretrained weights

vgg_model=tf.keras.Sequential()

#https://keras.io/api/applications/vgg/#vgg16-function
#https://keras.io/guides/transfer_learning/
#https://chroniclesofai.com/transfer-learning-with-keras-resnet-50/
pre_vgg_model=tf.keras.applications.VGG16(include_top=False,weights="imagenet",classes=10,pooling=max,input_shape=(32,32,3))
pre_vgg_model.trainable=False
vgg_model.add(pre_vgg_model)
vgg_model.add(tf.keras.layers.Flatten())
vgg_model.add(tf.keras.layers.Dense(512, activation='relu'))
vgg_model.add(tf.keras.layers.Dense(10, activation='softmax'))


2022-03-10 16:03:04.808216: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-10 16:03:05.178870: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-10 16:03:05.623496: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [28]:
vgg_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dense_5 (Dense)             (None, 10)                5130      
                                                                 
Total params: 14,982,474
Trainable params: 267,786
Non-trainable params: 14,714,688
_________________________________________________________________


In [29]:

# define optimizer (dp-sgd) 
vgg_optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)
# define loss function
vgg_loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [30]:
# Compile model
vgg_model.compile(optimizer=vgg_optimizer, loss=vgg_loss, metrics=['accuracy'])
# Train model
vgg_model.fit(x_train, y_train,
          epochs=epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size)


Train on 50000 samples, validate on 10000 samples


2022-03-10 16:03:18.918033: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-10 16:03:19.508836: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-10 16:03:19.670580: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/50


2022-03-10 16:03:19.988683: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - ETA: 0s - loss: 1.9535 - acc: 0.3352

2022-03-10 16:12:39.952061: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - 568s 11ms/sample - loss: 1.9535 - acc: 0.3352 - val_loss: 1.7484 - val_acc: 0.3937
Epoch 2/50
50000/50000 [==============================] - 508s 10ms/sample - loss: 1.6478 - acc: 0.4352 - val_loss: 1.5992 - val_acc: 0.4455
Epoch 3/50
50000/50000 [==============================] - 520s 10ms/sample - loss: 1.5467 - acc: 0.4633 - val_loss: 1.5284 - val_acc: 0.4649
Epoch 4/50
50000/50000 [==============================] - 525s 11ms/sample - loss: 1.4940 - acc: 0.4773 - val_loss: 1.4873 - val_acc: 0.4801
Epoch 5/50
50000/50000 [==============================] - 524s 10ms/sample - loss: 1.4583 - acc: 0.4924 - val_loss: 1.4691 - val_acc: 0.4873
Epoch 6/50
50000/50000 [==============================] - 529s 11ms/sample - loss: 1.4342 - acc: 0.5014 - val_loss: 1.4475 - val_acc: 0.4916
Epoch 7/50
50000/50000 [==============================] - 516s 10ms/sample - loss: 1.4157 - acc: 0.5084 - val_loss: 1.4292 - val_acc: 0.5010
Epoch 8/50
50000/50000 [

In [31]:
# noise_multiplier = FLAGS.noise_multiplier
# sampling_probability = FLAGS.batch_size / 60000
# steps = FLAGS.epochs * 60000 // FLAGS.batch_size

In [32]:
# from absl import app
# from absl import flags
# from absl import logging
# FLAGS = flags.FLAGS
orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
rdp = compute_rdp(q = batch_size / 50000,
                  noise_multiplier = noise_multiplier,
                  steps = epochs * 50000 // batch_size,
                  orders=orders)

In [33]:
epsilon = get_privacy_spent(orders, rdp, target_delta=1e-5)[0]
epsilon

2.9227129242856575

### Model3: HOG classification